In [1]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.patches as patches
import cmcrameri.cm as cmc
import random
from tqdm.notebook import tqdm
from geopy.distance import geodesic


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
creepmeters = pd.read_csv('../../Data/DATA_tidied/creepmeter_metadata_post_standardisation_sac_codes_updated.csv')
creepmeters.drop_duplicates('Creepmeter_abbrv',inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='XMBC'].index,inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='TABC'].index,inplace=True)
creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='CHP1'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='CTM1'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='XHR3'].index,inplace=True)
#creepmeters.drop(creepmeters[creepmeters['Creepmeter_abbrv']=='SH30'].index,inplace=True) #needs checking as defintely picks




creepmeters.reset_index(inplace=True,drop=True)
creepmeters_picks = pd.read_csv('../../Data/DATA_tidied/Picks/All_picks_29_May_2025.csv',index_col=0)
creepmeters_picks['Start Time'] = pd.to_datetime(creepmeters_picks['Start Time'])
creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Start Time']>dt.datetime(2024,1,1,0,0,0)].index,inplace=True)
creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Displacement, mm']<0.02].index,inplace=True)
networks = creepmeters['Network'].unique()
networks

/var/folders/56/47k8q3q1449cq8kwzkz5r8ch0000gr/T/ipykernel_12026/2006262863.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  creepmeters_picks['Start Time'] = pd.to_datetime(creepmeters_picks['Start Time'])


array(['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL',
       'RID', 'CAL'], dtype=object)

In [3]:
def compute_epicentral_distance_and_travel_time(row):
    """
    Computes the epicentral distance and surface wave travel time between
    an earthquake and a creepmeter.

    Parameters:
        row: A row from the nearest_eq_df DataFrame, containing earthquake and creepmeter coordinates.

    Returns:
        A tuple with:
        - Epicentral distance in kilometres
        - Estimated surface wave travel time in seconds (assuming 3.5 km/s)
    """
    epicentral_distance_km = geodesic(
        (row['Latitude_creep'], row['Longitude_creep']),
        (row['latitude'], row['longitude'])
    ).km
    surface_wave_speed_km_s = 3.5  # km/s for Rayleigh/Love waves in crust
    travel_time_s = epicentral_distance_km / surface_wave_speed_km_s
    return epicentral_distance_km, travel_time_s


In [4]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from geopy.distance import geodesic

# Define parameters
radius_km = 1000
maxradius_deg = radius_km / 111.0
min_magnitude = 4  # Set minimum magnitude for EQ search

# Containers to store all output across networks
my_data = pd.DataFrame()
percentage_triggered_all = pd.DataFrame()

# Loop over each network in your list
for j in range(len(networks)):
    print(networks[j])

    # Limit processing to specific networks of interest
    if networks[j] in ['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL', 'RID', 'CAL']:

        # Filter creepmeters to just those in this network
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network'] != networks[j]].index)

        # Loop over each creepmeter in the network
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)

            # Filter picks (creep events) for this creepmeter
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation'] != abbrv].index)

            # Skip if no picks for this creepmeter
            if picks.empty:
                continue

            # Get creepmeter location
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv, lat, long)

            # Extract per-event sampling interval (in seconds)
            sampling_freq = picks['Sampling rate, s']

            # Determine query time window from picks
            starttime = UTCDateTime((pd.to_datetime(picks['Start Time']).min() - pd.DateOffset(months=1)).strftime('%Y-%m-%d'))
            endtime = UTCDateTime((pd.to_datetime(picks['Start Time']).max() + pd.DateOffset(months=1)).strftime('%Y-%m-%d'))

            # Query IRIS FDSN event service
            client = Client("IRIS")
            try:
                catalog = client.get_events(
                    starttime=starttime,
                    endtime=endtime,
                    latitude=lat,
                    longitude=long,
                    maxradius=maxradius_deg,  # ~500 km
                    minmagnitude=min_magnitude
                )
            except Exception as e:
                print(f"Failed to retrieve events for {abbrv}: {e}")
                continue

            # Parse earthquake metadata into DataFrame
            eq_data = []
            for event in catalog:
                origin = event.preferred_origin() or event.origins[0]
                magnitude = event.preferred_magnitude() or event.magnitudes[0]
                eq_data.append({
                    'eq_time': origin.time.datetime,
                    'magnitude': magnitude.mag,
                    'latitude': origin.latitude,
                    'longitude': origin.longitude,
                    'depth_km': origin.depth / 1000.0,
                    'place': event.resource_id.id.split('/')[-1]
                })

            eq_df = pd.DataFrame(eq_data)
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Search for plausible EQ triggers based on wave arrival time
            expanded_matches = []
            for idx, pick in picks.iterrows():
                start_time = pick['Start Time']
                sampling_interval = pick['Sampling rate, s']

                valid_trigger_found = False
                best_match = None
                best_time_diff = np.inf

                for _, eq in eq_df.iterrows():
                    dist_km = geodesic((lat, long), (eq['latitude'], eq['longitude'])).km
                    travel_time = dist_km / 3.5
                    time_since_eq = (start_time - eq['eq_time']).total_seconds()
                    upper_bound = travel_time + 2 * sampling_interval + 1.0 * sampling_interval + 60

                    if travel_time <= time_since_eq <= upper_bound:
                        if time_since_eq < best_time_diff:
                            valid_trigger_found = True
                            best_time_diff = time_since_eq
                            best_match = {
                                'Start Time': start_time,
                                'abbrv': abbrv,
                                'eq_time': eq['eq_time'],
                                'magnitude': eq['magnitude'],
                                'latitude': eq['latitude'],
                                'longitude': eq['longitude'],
                                'depth_km': eq['depth_km'],
                                'place': eq['place'],
                                'Time Since EQ [s]': time_since_eq,
                                'Surface Wave Travel Time [s]': travel_time,
                                'Upper Tolerance Bound [s]': upper_bound,
                                'Likely Triggered': True,
                                'Sampling Interval [s]': sampling_interval,
                                'Latitude_creep': lat,
                                'Longitude_creep': long
                            }

                if not valid_trigger_found:
                    best_match = {
                        'Start Time': start_time,
                        'abbrv': abbrv,
                        'eq_time': pd.NaT,
                        'magnitude': np.nan,
                        'latitude': np.nan,
                        'longitude': np.nan,
                        'depth_km': np.nan,
                        'place': None,
                        'Time Since EQ [s]': np.nan,
                        'Surface Wave Travel Time [s]': np.nan,
                        'Upper Tolerance Bound [s]': np.nan,
                        'Likely Triggered': False,
                        'Sampling Interval [s]': sampling_interval,
                        'Latitude_creep': lat,
                        'Longitude_creep': long
                    }

                expanded_matches.append(best_match)

            nearest_eq_df = pd.DataFrame(expanded_matches)

            # Compute percentage of events considered valid triggers
            count_valid = nearest_eq_df['Likely Triggered'].sum()
            percent_valid = 100 * count_valid / len(picks)

            print(f"{percent_valid:.2f}% of {len(picks)} events have valid earthquake triggers (wave arrival + resolution buffer).")

            percentage_triggered = pd.DataFrame({
                'Creepmeter': [abbrv],
                'percentage valid triggers': [round(percent_valid, 2)],
                'Latitude': lat
            })

            creepmeter_eq_picks = pd.concat([picks.reset_index(drop=True), nearest_eq_df.reset_index(drop=True)], axis=1)
            my_data = pd.concat([my_data, creepmeter_eq_picks])
            percentage_triggered_all = pd.concat([percentage_triggered_all, percentage_triggered])

        percentage_triggered_all.sort_values(by='Latitude', inplace=True, ascending=False)
        percentage_triggered_all.reset_index(inplace=True, drop=True)

my_data.to_csv("triggered_creep_events_matched_to_nearest_EQ.csv", index=False)
percentage_triggered_all.to_csv("triggered_creep_percentage_summary.csv", index=False)


EAF
BAL1
BAL1 37.9903 38.199
0.00% of 2 events have valid earthquake triggers (wave arrival + resolution buffer).
GOK1
GOK1 38.006 36.5267
0.00% of 130 events have valid earthquake triggers (wave arrival + resolution buffer).
GOZ1
GOZ1 38.1759 38.0103
0.00% of 1 events have valid earthquake triggers (wave arrival + resolution buffer).
HAS1
HAS1 36.8006 36.5185
0.00% of 8 events have valid earthquake triggers (wave arrival + resolution buffer).
HAT1
HAT1 38.387 36.2803
0.00% of 7 events have valid earthquake triggers (wave arrival + resolution buffer).
KAR1
KIR1
KIR1 36.4791 36.3339
0.00% of 6 events have valid earthquake triggers (wave arrival + resolution buffer).
ORM1
ORM1 38.2113 38.7732
0.00% of 5 events have valid earthquake triggers (wave arrival + resolution buffer).
PAN1
PAN1 38.699 39.9537
5.00% of 20 events have valid earthquake triggers (wave arrival + resolution buffer).
PAS1
PAS1 38.699 39.9537
0.00% of 16 events have valid earthquake triggers (wave arrival + resolution bu

KeyboardInterrupt: 

In [5]:
picks

,Network,Creepmeter full name,Creepmeter abbreviation,Latitude,Longitude,"Sampling rate, s",Start Time,End Time,"Displacement, mm","Duration, hrs","Maximum velocity, m/s",Event_id
280,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,1989-03-23 01:00:00,24/3/89 16:50,1.53,39.833333,6.666667e-08,0280_CRR1
281,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,1989-03-24 17:30:00,26/3/89 18:49,0.56,49.333056,3.333333e-08,0281_CRR1
425,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,1991-03-19 01:09:00,19/3/91 16:39,0.21,15.500000,3.333333e-08,0425_CRR1
430,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,1991-03-20 07:50:00,21/3/91 04:59,0.14,21.166389,6.666667e-08,0430_CRR1
536,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,1992-02-13 23:39:00,15/2/92 16:20,0.29,40.666944,3.333333e-08,0536_CRR1
...,...,...,...,...,...,...,...,...,...,...,...,...
3440,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,2011-12-25 00:20:00,25/12/11 17:19,0.27,16.999722,3.333333e-08,3440_CRR1
3444,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,2012-01-01 21:00:00,3/1/12 11:10,0.85,38.166667,1.500000e-07,3444_CRR1
3564,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,2013-02-14 00:49:00,18/2/13 19:00,2.43,114.166944,1.000000e-07,3564_CRR1
3659,PARK,Carr Ranch,CRR1,35.835,-120.363,600.0,2014-01-25 09:00:00,26/1/14 17:30,0.73,32.500000,8.333333e-08,3659_CRR1


In [13]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from geopy.distance import geodesic
import os
import pandas as pd

# Define parameters
radius_km = 500
maxradius_deg = radius_km / 111.0
min_magnitude = 4  # Set minimum magnitude for EQ search

# Channels to try
channels_to_try = ["BHZ", "BHE", "BHN"]

# Containers to store all output across networks
my_data = pd.DataFrame()
percentage_triggered_all = pd.DataFrame()

# Function to download waveform from closest broadband station with fallback

def download_waveform_for_event(event_time, eq_lat, eq_lon, event_id, creep_lat, creep_lon,
                                 duration=300, save_folder="waveforms", radius_deg=1.0):
    client = Client("IRIS")
    starttime = UTCDateTime(event_time)
    endtime = starttime + duration

    try:
        inventory = client.get_stations(
            latitude=creep_lat,
            longitude=creep_lon,
            maxradius=radius_deg,
            channel="BH?",
            level="channel",
            starttime=starttime,
            endtime=endtime
        )

        if not inventory or len(inventory) == 0:
            print(f"No stations found near {event_id}")
            return

        # Create list of candidate stations sorted by distance
        station_list = []
        for net in inventory:
            for sta in net:
                sta_lat = sta.latitude
                sta_lon = sta.longitude
                dist_km = geodesic((creep_lat, creep_lon), (sta_lat, sta_lon)).km
                station_list.append({
                    "distance": dist_km,
                    "network": net.code,
                    "station": sta.code,
                    "location": "*"
                })

        station_list.sort(key=lambda x: x["distance"])

        # Try stations in order of proximity and download all BH? channels from the first working station
        for sta in station_list:
            try:
                print(f"Trying station {sta['network']}.{sta['station']} ({sta['distance']:.1f} km) for all BH? channels")
                st = client.get_waveforms(
                    network=sta["network"],
                    station=sta["station"],
                    location=sta["location"],
                    channel="BH?",
                    starttime=starttime,
                    endtime=endtime
                )
                os.makedirs(save_folder, exist_ok=True)
                output_path = os.path.join(save_folder, f"{event_id}.mseed")
                st.write(output_path, format="MSEED")
                print(f"Saved waveform to {output_path}")
                return
            except Exception as inner_e:
                print(f"Station {sta['network']}.{sta['station']} failed: {inner_e}")

        print(f"All station attempts failed for event {event_id}.")

    except Exception as e:
        print(f"Failed to download waveform for {event_id}: {e}")

# Loop over each network in your list
for j in range(len(networks)):
    print(networks[j])

    # Limit processing to specific networks of interest
    if networks[j] in ['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL', 'RID', 'CAL']:

        # Filter creepmeters to just those in this network
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network'] != networks[j]].index)

        # Loop over each creepmeter in the network
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)

            # Filter picks (creep events) for this creepmeter
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation'] != abbrv].index)

            # Skip if no picks for this creepmeter
            if picks.empty:
                continue

            # Get creepmeter location
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv, lat, long)

            # Extract per-event sampling interval (in seconds)
            sampling_freq = picks['Sampling rate, s']

            # Determine query time window from picks
            starttime = UTCDateTime((pd.to_datetime(picks['Start Time']).min() - pd.DateOffset(months=1)).strftime('%Y-%m-%d'))
            endtime = UTCDateTime((pd.to_datetime(picks['Start Time']).max() + pd.DateOffset(months=1)).strftime('%Y-%m-%d'))

            # Query IRIS FDSN event service
            client = Client("IRIS")
            try:
                catalog = client.get_events(
                    starttime=starttime,
                    endtime=endtime,
                    latitude=lat,
                    longitude=long,
                    maxradius=maxradius_deg,  # ~500 km
                    minmagnitude=min_magnitude
                )
            except Exception as e:
                print(f"Failed to retrieve events for {abbrv}: {e}")
                continue

            # Parse earthquake metadata into DataFrame
            eq_data = []
            for event in catalog:
                origin = event.preferred_origin() or event.origins[0]
                magnitude = event.preferred_magnitude() or event.magnitudes[0]
                eq_data.append({
                    'eq_time': origin.time.datetime,
                    'magnitude': magnitude.mag,
                    'latitude': origin.latitude,
                    'longitude': origin.longitude,
                    'depth_km': origin.depth / 1000.0,
                    'place': event.resource_id.id.split('/')[-1]
                })

            eq_df = pd.DataFrame(eq_data)
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Search for plausible EQ triggers based on wave arrival time
            expanded_matches = []
            for idx, pick in picks.iterrows():
                start_time = pick['Start Time']
                sampling_interval = pick['Sampling rate, s']
                pick_event_id = pick['Event_id']

                valid_trigger_found = False
                best_match = None
                best_time_diff = np.inf

                for _, eq in eq_df.iterrows():
                    dist_km = geodesic((lat, long), (eq['latitude'], eq['longitude'])).km
                    travel_time = dist_km / 3.5
                    time_since_eq = (start_time - eq['eq_time']).total_seconds()
                    upper_bound = travel_time + 2 * sampling_interval + 1.0 * sampling_interval + 60

                    if travel_time <= time_since_eq <= upper_bound:
                        if time_since_eq < best_time_diff:
                            valid_trigger_found = True
                            best_time_diff = time_since_eq
                            best_match = {
                                'Start Time': start_time,
                                'abbrv': abbrv,
                                'eq_time': eq['eq_time'],
                                'magnitude': eq['magnitude'],
                                'latitude': eq['latitude'],
                                'longitude': eq['longitude'],
                                'depth_km': eq['depth_km'],
                                'place': eq['place'],
                                'Time Since EQ [s]': time_since_eq,
                                'Surface Wave Travel Time [s]': travel_time,
                                'Upper Tolerance Bound [s]': upper_bound,
                                'Likely Triggered': True,
                                'Sampling Interval [s]': sampling_interval,
                                'Latitude_creep': lat,
                                'Longitude_creep': long,
                                'event_id_from_picks': pick_event_id
                            }

                if not valid_trigger_found:
                    best_match = {
                        'Start Time': start_time,
                        'abbrv': abbrv,
                        'eq_time': pd.NaT,
                        'magnitude': np.nan,
                        'latitude': np.nan,
                        'longitude': np.nan,
                        'depth_km': np.nan,
                        'place': None,
                        'Time Since EQ [s]': np.nan,
                        'Surface Wave Travel Time [s]': np.nan,
                        'Upper Tolerance Bound [s]': np.nan,
                        'Likely Triggered': False,
                        'Sampling Interval [s]': sampling_interval,
                        'Latitude_creep': lat,
                        'Longitude_creep': long,
                        'event_id_from_picks': pick_event_id
                    }

                expanded_matches.append(best_match)

                # If valid, download waveform
                if best_match['Likely Triggered']:
                    event_id = best_match['event_id_from_picks']
                    eq_lat = best_match['latitude']
                    eq_lon = best_match['longitude']
                    eq_time = best_match['eq_time']
                    download_waveform_for_event(eq_time, eq_lat, eq_lon, event_id, lat, long)

            nearest_eq_df = pd.DataFrame(expanded_matches)

            # Compute percentage of events considered valid triggers
            count_valid = nearest_eq_df['Likely Triggered'].sum()
            percent_valid = 100 * count_valid / len(picks)

            print(f"{percent_valid:.2f}% of {len(picks)} events have valid earthquake triggers (wave arrival + resolution buffer).")

            percentage_triggered = pd.DataFrame({
                'Creepmeter': [abbrv],
                'percentage valid triggers': [round(percent_valid, 2)],
                'Latitude': lat
            })

            creepmeter_eq_picks = pd.concat([picks.reset_index(drop=True), nearest_eq_df.reset_index(drop=True)], axis=1)
            my_data = pd.concat([my_data, creepmeter_eq_picks])
            percentage_triggered_all = pd.concat([percentage_triggered_all, percentage_triggered])

        percentage_triggered_all.sort_values(by='Latitude', inplace=True, ascending=False)
        percentage_triggered_all.reset_index(inplace=True, drop=True)

my_data.to_csv("triggered_creep_events_matched_to_nearest_EQ.csv", index=False)
percentage_triggered_all.to_csv("triggered_creep_percentage_summary.csv", index=False)


EAF
BAL1
BAL1 37.9903 38.199
0.00% of 2 events have valid earthquake triggers (wave arrival + resolution buffer).
GOK1
GOK1 38.006 36.5267
0.00% of 130 events have valid earthquake triggers (wave arrival + resolution buffer).
GOZ1
GOZ1 38.1759 38.0103
0.00% of 1 events have valid earthquake triggers (wave arrival + resolution buffer).
HAS1
HAS1 36.8006 36.5185
0.00% of 8 events have valid earthquake triggers (wave arrival + resolution buffer).
HAT1
HAT1 38.387 36.2803
0.00% of 7 events have valid earthquake triggers (wave arrival + resolution buffer).
KAR1
KIR1
KIR1 36.4791 36.3339
0.00% of 6 events have valid earthquake triggers (wave arrival + resolution buffer).
ORM1
ORM1 38.2113 38.7732
0.00% of 5 events have valid earthquake triggers (wave arrival + resolution buffer).
PAN1
PAN1 38.699 39.9537
Failed to download waveform for 4700_PAN1: No data available for request.
HTTP Status code: 204
Detailed response of server:


5.00% of 20 events have valid earthquake triggers (wave arrival

KeyboardInterrupt: 

In [15]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from geopy.distance import geodesic
import os
import pandas as pd
import numpy as np

# Define parameters
radius_km = 500
maxradius_deg = radius_km / 111.0
min_magnitude = 4  # Set minimum magnitude for EQ search

# Channels to try
channels_to_try = ["BHZ", "BHE", "BHN"]

# Containers to store all output across networks
my_data = pd.DataFrame()
percentage_triggered_all = pd.DataFrame()

# Function to download waveform from closest broadband station with fallback
def download_waveform_for_event(event_time, eq_lat, eq_lon, event_id, creep_lat, creep_lon,
                                 duration=300, save_folder="waveforms", radius_deg=1.0):
    client = Client("IRIS")
    starttime = UTCDateTime(event_time)
    endtime = starttime + duration

    try:
        inventory = client.get_stations(
            latitude=creep_lat,
            longitude=creep_lon,
            maxradius=radius_deg,
            channel="BH?",
            level="channel",
            starttime=starttime,
            endtime=endtime
        )

        if not inventory or len(inventory) == 0:
            print(f"No stations found near {event_id}")
            return

        # Create list of candidate stations sorted by distance
        station_list = []
        for net in inventory:
            for sta in net:
                sta_lat = sta.latitude
                sta_lon = sta.longitude
                dist_km = geodesic((creep_lat, creep_lon), (sta_lat, sta_lon)).km
                station_list.append({
                    "distance": dist_km,
                    "network": net.code,
                    "station": sta.code,
                    "location": "*"
                })

        station_list.sort(key=lambda x: x["distance"])

        # Try stations in order of proximity and download all BH? channels from the first working station
        for sta in station_list:
            try:
                print(f"Trying station {sta['network']}.{sta['station']} ({sta['distance']:.1f} km) for all BH? channels")
                st = client.get_waveforms(
                    network=sta["network"],
                    station=sta["station"],
                    location=sta["location"],
                    channel="BH?",
                    starttime=starttime,
                    endtime=endtime
                )
                os.makedirs(save_folder, exist_ok=True)
                output_path = os.path.join(save_folder, f"{event_id}.mseed")
                st.write(output_path, format="MSEED")
                print(f"Saved waveform to {output_path}")
                return
            except Exception as inner_e:
                print(f"Station {sta['network']}.{sta['station']} failed: {inner_e}")

        print(f"All station attempts failed for event {event_id}.")

    except Exception as e:
        print(f"Failed to download waveform for {event_id}: {e}")

# Define FDSN clients for global and regional catalogs
global_client = Client("IRIS")
regional_client = Client("NCEDC")  # Northern California Earthquake Data Center

# Loop over each network in your list
for j in range(len(networks)):
    print(networks[j])

    # Limit processing to specific networks of interest
    if networks[j] in ['HOL','HAY','SOCAL','PARK']:#['EAF', 'PARK', 'UTA', 'NAF', 'HAY', 'CHAF', 'HOL', 'DSF', 'SOCAL', 'RID', 'CAL']:

        # Filter creepmeters to just those in this network
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network'] != networks[j]].index)

        # Loop over each creepmeter in the network
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)

            # Filter picks (creep events) for this creepmeter
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation'] != abbrv].index)

            # Skip if no picks for this creepmeter
            if picks.empty:
                continue

            # Get creepmeter location
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv, lat, long)

            # Extract per-event sampling interval (in seconds)
            sampling_freq = picks['Sampling rate, s']

            # Determine query time window from picks
            starttime = UTCDateTime((pd.to_datetime(picks['Start Time']).min() - pd.DateOffset(months=1)).strftime('%Y-%m-%d'))
            endtime = UTCDateTime((pd.to_datetime(picks['Start Time']).max() + pd.DateOffset(months=1)).strftime('%Y-%m-%d'))

            try:
                # Query global catalog
                global_catalog = global_client.get_events(
                    starttime=starttime,
                    endtime=endtime,
                    latitude=lat,
                    longitude=long,
                    maxradius=maxradius_deg,  # ~500 km
                    minmagnitude=min_magnitude
                )

                # Query regional catalog
                regional_catalog = regional_client.get_events(
                    starttime=starttime,
                    endtime=endtime,
                    latitude=lat,
                    longitude=long,
                    maxradius=maxradius_deg,
                    minmagnitude=min_magnitude
                )

                # Combine events from both catalogs
                combined_events = list(global_catalog) + list(regional_catalog)

                # Deduplicate events by origin time and rounded lat/lon (4 decimals)
                unique_events_dict = {}
                for event in combined_events:
                    origin = event.preferred_origin() or event.origins[0]
                    key = (
                        origin.time.strftime("%Y-%m-%dT%H:%M:%S.%f"),
                        round(origin.latitude, 4),
                        round(origin.longitude, 4)
                    )
                    unique_events_dict[key] = event

                combined_catalog = list(unique_events_dict.values())

            except Exception as e:
                print(f"Failed to retrieve events for {abbrv}: {e}")
                continue

            # Parse earthquake metadata into DataFrame
            eq_data = []
            for event in combined_catalog:
                origin = event.preferred_origin() or event.origins[0]
                magnitude = event.preferred_magnitude() or event.magnitudes[0]
                eq_data.append({
                    'eq_time': origin.time.datetime,
                    'magnitude': magnitude.mag,
                    'latitude': origin.latitude,
                    'longitude': origin.longitude,
                    'depth_km': origin.depth / 1000.0,
                    'place': event.resource_id.id.split('/')[-1]
                })

            eq_df = pd.DataFrame(eq_data)
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Search for plausible EQ triggers based on wave arrival time
            expanded_matches = []
            for idx, pick in picks.iterrows():
                start_time = pick['Start Time']
                sampling_interval = pick['Sampling rate, s']
                pick_event_id = pick['Event_id']

                valid_trigger_found = False
                best_match = None
                best_time_diff = np.inf

                for _, eq in eq_df.iterrows():
                    dist_km = geodesic((lat, long), (eq['latitude'], eq['longitude'])).km
                    travel_time = dist_km / 3.5
                    time_since_eq = (start_time - eq['eq_time']).total_seconds()
                    upper_bound = travel_time + 2 * sampling_interval + 1.0 * sampling_interval + 60

                    if travel_time <= time_since_eq <= upper_bound:
                        if time_since_eq < best_time_diff:
                            valid_trigger_found = True
                            best_time_diff = time_since_eq
                            best_match = {
                                'Start Time': start_time,
                                'abbrv': abbrv,
                                'eq_time': eq['eq_time'],
                                'magnitude': eq['magnitude'],
                                'latitude': eq['latitude'],
                                'longitude': eq['longitude'],
                                'depth_km': eq['depth_km'],
                                'place': eq['place'],
                                'Time Since EQ [s]': time_since_eq,
                                'Surface Wave Travel Time [s]': travel_time,
                                'Upper Tolerance Bound [s]': upper_bound,
                                'Likely Triggered': True,
                                'Sampling Interval [s]': sampling_interval,
                                'Latitude_creep': lat,
                                'Longitude_creep': long,
                                'event_id_from_picks': pick_event_id
                            }

                if not valid_trigger_found:
                    best_match = {
                        'Start Time': start_time,
                        'abbrv': abbrv,
                        'eq_time': pd.NaT,
                        'magnitude': np.nan,
                        'latitude': np.nan,
                        'longitude': np.nan,
                        'depth_km': np.nan,
                        'place': None,
                        'Time Since EQ [s]': np.nan,
                        'Surface Wave Travel Time [s]': np.nan,
                        'Upper Tolerance Bound [s]': np.nan,
                        'Likely Triggered': False,
                        'Sampling Interval [s]': sampling_interval,
                        'Latitude_creep': lat,
                        'Longitude_creep': long,
                        'event_id_from_picks': pick_event_id
                    }

                expanded_matches.append(best_match)

                # If valid, download waveform
                if best_match['Likely Triggered']:
                    event_id = best_match['event_id_from_picks']
                    eq_lat = best_match['latitude']
                    eq_lon = best_match['longitude']
                    eq_time = best_match['eq_time']
                    download_waveform_for_event(eq_time, eq_lat, eq_lon, event_id, lat, long)

            nearest_eq_df = pd.DataFrame(expanded_matches)

            # Compute percentage of events considered valid triggers
            count_valid = nearest_eq_df['Likely Triggered'].sum()
            percent_valid = 100 * count_valid / len(picks)

            print(f"{percent_valid:.2f}% of {len(picks)} events have valid earthquake triggers (wave arrival + resolution buffer).")

            percentage_triggered = pd.DataFrame({
                'Creepmeter': [abbrv],
                'percentage valid triggers': [round(percent_valid, 2)],
                'Latitude': lat
            })

            creepmeter_eq_picks = pd.concat([picks.reset_index(drop=True), nearest_eq_df.reset_index(drop=True)], axis=1)
            my_data = pd.concat([my_data, creepmeter_eq_picks])
            percentage_triggered_all = pd.concat([percentage_triggered_all, percentage_triggered])

        percentage_triggered_all.sort_values(by='Latitude', inplace=True, ascending=False)
        percentage_triggered_all.reset_index(inplace=True, drop=True)

# Save outputs
my_data.to_csv("triggered_creep_events_matched_to_nearest_EQ.csv", index=False)
percentage_triggered_all.to_csv("triggered_creep_percentage_summary.csv", index=False)


EAF
PARK
C461
C461 35.724 -120.282
0.00% of 43 events have valid earthquake triggers (wave arrival + resolution buffer).
C462
C462 35.724 -120.282
0.00% of 24 events have valid earthquake triggers (wave arrival + resolution buffer).
CRR1
CRR1 35.835 -120.363
0.00% of 139 events have valid earthquake triggers (wave arrival + resolution buffer).
WKR1
WKR1 35.858 -120.392
Trying station YH.CRAK (16.0 km) for all BH? channels
Saved waveform to waveforms/2503_WKR1.mseed
0.63% of 158 events have valid earthquake triggers (wave arrival + resolution buffer).
X461
X461 35.723 -120.278
Trying station BK.PKD (34.3 km) for all BH? channels
Saved waveform to waveforms/2073_X461.mseed
Trying station BK.PKD (34.3 km) for all BH? channels
Saved waveform to waveforms/2075_X461.mseed
1.37% of 146 events have valid earthquake triggers (wave arrival + resolution buffer).
XGH1
XGH1 35.82 -120.348
0.00% of 88 events have valid earthquake triggers (wave arrival + resolution buffer).
XHSW
XHSW 35.862 -120.42


KeyboardInterrupt: 

Other things for later

In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_rainfall = pd.DataFrame()
for j in range(len(networks)):
    print(networks[j])
    if networks[j] in ['HOL']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)
            rainfall = pd.read_csv('../../Data/ECMWF/Creepmeter_precipitation/{p}/Precipitation_ECMWF_{q}_02_JUL_2024.csv'.format(p=networks[j],q=abbrv))

            # Ensure datetime format and sort
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            rainfall['rain_time'] = pd.to_datetime(rainfall['Time (UTC)'])

            picks = picks.sort_values('Start Time').reset_index(drop=True)
            rainfall = rainfall.sort_values('rain_time').reset_index(drop=True)

            # Set rain_time as index for fast slicing
            rainfall = rainfall.set_index('Time (UTC)')

            # Ensure the index is sorted
            rainfall = rainfall.sort_index()

            def rain_in_previous_hour(pick_time):
                pick_time = pd.to_datetime(pick_time)
                start = pick_time - pd.Timedelta(hours=1)
                # Clip start and end to be within the rainfall index range
                rainfall.index = pd.to_datetime(rainfall.index)
                start = max(start, rainfall.index.min())
                end = min(pick_time, rainfall.index.max())

                rain_window = rainfall.loc[start:end]
                if rain_window.empty:
                    return 0.0
                else:
                    return rain_window['Precipitation, m'].sum()

            picks['rain_last_hour'] = picks['Start Time'].apply(rain_in_previous_hour)
            picks['rain_before'] = picks['rain_last_hour'] > 0.0005
            
            if abbrv in ['XHR2']:
                boolarr = picks['rain_before']==True
                print(picks[boolarr])


            # Count how many are within 1 day
            count_within_hour = picks['rain_before'].sum()
    

            # Total number of entries
            total_count = len(picks)

            # Percentage
            percent_within_hour = 100 * count_within_hour / total_count

            #print(f"{percent_within_day:.2f}% of {total_count} events occur within 1 day of an earthquake.")
            print(f"{percent_within_hour:.2f}% of {total_count} events occur within 1 hour of 0.5mm of rainfall.")

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'percentage in hour':[round(percent_within_hour,2)],
                                                    'Latitude':lat})


            percentage_triggered_all_rainfall = pd.concat([percentage_triggered_all_rainfall,percentage_triggered])
        percentage_triggered_all_rainfall.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_rainfall.reset_index(inplace=True,drop=True)








In [ ]:
cmap=cmc.managua

In [ ]:
colours  = cmap(np.linspace(0, 1, 10))

In [ ]:
plt.figure(figsize=(18,6))
ax = plt.subplot(1,1,1)
#plt.plot(percentage_triggered_all['Latitude'],percentage_triggered_all['percentage in day'])
# Add a rectangle: (x, y) = lower left corner; width and height
rect_Hawyard = patches.Rectangle((-0.5, 0), width=3, height=60, linewidth=2, edgecolor=colours[0], facecolor=colours[0],alpha=0.2)
rect_Calaveras = patches.Rectangle((2.5, 0), width=1, height=60, linewidth=2, edgecolor=colours[1], facecolor=colours[1],alpha=0.2)
rect_Hollister = patches.Rectangle((3.5, 0), width=10, height=60, linewidth=2, edgecolor=colours[2], facecolor=colours[2],alpha=0.2)
rect_Parkfield = patches.Rectangle((13.5, 0), width=15, height=60, linewidth=2, edgecolor=colours[4], facecolor=colours[4],alpha=0.2)
rect_SoCal = patches.Rectangle((28.5, 0), width=10, height=60, linewidth=2, edgecolor=colours[6], facecolor=colours[6],alpha=0.2)


plt.xticks(np.arange(len(percentage_triggered_all)),percentage_triggered_all['Creepmeter'],rotation=90)
# Add the patch to the Axes
ax.add_patch(rect_Hawyard)
ax.add_patch(rect_Calaveras)
ax.add_patch(rect_Hollister)
ax.add_patch(rect_Parkfield)
ax.add_patch(rect_SoCal)

plt.plot(np.arange(len(percentage_triggered_all)),percentage_triggered_all['percentage in hour'],
         color=colours[3],marker='o',label='Earthquake in hour prior')
plt.plot(np.arange(len(percentage_triggered_all_rainfall)),percentage_triggered_all_rainfall['percentage in hour'],
         color=colours[5],marker='^',label='Rainfall in hour prior')
plt.ylabel("% of events")
plt.xlabel('Creepmeter')
plt.legend()
plt.show()


In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_rainfall_boot = pd.DataFrame()
for j in range(len(networks)):
    print(networks[j])
    if networks[j] in ['SOCAL','HAY','HOL','CAL','PARK']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)
            rainfall = pd.read_csv('../../Data/ECMWF/Creepmeter_precipitation/{p}/Precipitation_ECMWF_{q}_02_JUL_2024.csv'.format(p=networks[j],q=abbrv))

            # Ensure datetime format and sort
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            rainfall['rain_time'] = pd.to_datetime(rainfall['Time (UTC)'])

            picks = picks.sort_values('Start Time').reset_index(drop=True)
            rainfall = rainfall.sort_values('rain_time').reset_index(drop=True)

            # Set rain_time as index for fast slicing
            rainfall = rainfall.set_index('Time (UTC)')

            # Ensure the index is sorted
            rainfall = rainfall.sort_index()

            def rain_in_previous_hour(pick_time):
                pick_time = pd.to_datetime(pick_time)
                start = pick_time - pd.Timedelta(hours=1)
                #start = pick_time - pd.Timedelta(days = 1)
                # Clip start and end to be within the rainfall index range
                rainfall.index = pd.to_datetime(rainfall.index)
                start = max(start, rainfall.index.min())
                end = min(pick_time, rainfall.index.max())

                rain_window = rainfall.loc[start:end]
                if rain_window.empty:
                    return 0.0
                else:
                    return rain_window['Precipitation, m'].sum()

            picks['rain_last_hour'] = picks['Start Time'].apply(rain_in_previous_hour)
            picks['rain_before'] = picks['rain_last_hour'] > 0.0005
            
            # Total number of entries
            total_count = len(picks)
            
            bootstrapped = []
            for m in tqdm(range(1000)):
                selected = []
                for n in range(len(picks)):
                    rand_index = random.randint(0, len(picks) - 1)
                    selected.append(picks['rain_before'].iloc[rand_index])
                
                # Count how many are within 1 day
                count_within_hour = np.sum(selected)            

                # Percentage
                percent_within_hour = 100 * (count_within_hour / len(picks))
                bootstrapped.append(percent_within_hour)


            per_85 = np.percentile(bootstrapped,85)
            per_15 = np.percentile(bootstrapped,15)
            median = np.median(bootstrapped)

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'per_85':[per_85],'per_15':[per_15],'median':[median],'Latitude':lat})


            percentage_triggered_all_rainfall_boot = pd.concat([percentage_triggered_all_rainfall_boot,percentage_triggered])
        percentage_triggered_all_rainfall_boot.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_rainfall_boot.reset_index(inplace=True,drop=True)

In [ ]:
percentage_triggered_all_rainfall_boot

In [ ]:
count_within_hour/total_count

In [ ]:
my_data = pd.DataFrame()
percentage_triggered_all_eq_boot = pd.DataFrame()
for j in tqdm(range(len(networks))):
    print(networks[j])
    if networks[j] in ['PARK','SOCAL','HAY','HOL','CAL']:
        creepmeters_network = creepmeters.drop(creepmeters[creepmeters['Network']!=networks[j]].index)
        for i in range(len(creepmeters_network)):#len(creepmeters)):
            abbrv = creepmeters_network['Creepmeter_abbrv'].iloc[i]
            print(abbrv)
            
            picks = creepmeters_picks.drop(creepmeters_picks[creepmeters_picks['Creepmeter abbreviation']!=abbrv].index)
            lat = creepmeters_network['Latitude'].iloc[i]
            long = creepmeters_network['Longitude'].iloc[i]
            print(abbrv,lat,long)


            # USGS API query parameters
            url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
            params = {
                "format": "geojson",
                "starttime": "1980-01-01",
                "endtime": "2024-01-01",
                "latitude": lat,
                "longitude": long,
                "maxradiuskm": 500,
                "minmagnitude": 4
            }

            response = requests.get(url, params=params)
            data = response.json()

            # Extract earthquake metadata
            eq_data = []
            for feature in data['features']:
                time = datetime.utcfromtimestamp(feature['properties']['time'] / 1000.0)
                magnitude = feature['properties']['mag']
                place = feature['properties']['place']
                lon_eq, lat_eq, depth = feature['geometry']['coordinates']  # [lon, lat, depth]
                eq_data.append({
                    'eq_time': time,
                    'magnitude': magnitude,
                    'latitude': lat_eq,
                    'longitude': lon_eq,
                    'depth_km': depth,
                    'place': place
                })

            # Create DataFrame
            eq_df = pd.DataFrame(eq_data)


            # Ensure both time columns are datetime
            picks['Start Time'] = pd.to_datetime(picks['Start Time'])
            eq_df['eq_time'] = pd.to_datetime(eq_df['eq_time'])

            # Sort both by time (important!)
            picks = picks.sort_values('Start Time').reset_index(drop=True)
            eq_df = eq_df.sort_values('eq_time').reset_index(drop=True)

            # Step 1: Use merge_asof twice — once backward and once forward
            df_before = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='backward', suffixes=('', '_before'))
            # Forward merge
            df_after = pd.merge_asof(picks, eq_df, left_on='Start Time', right_on='eq_time', direction='forward')

            # Rename the columns to prevent confusion
            df_after = df_after.rename(columns={
                'eq_time': 'eq_time_after',
                'magnitude': 'magnitude_after',
                'latitude': 'latitude_after',
                'longitude': 'longitude_after',
                'depth_km': 'depth_km_after',
                'place': 'place_after'
            })


            # Step 2: Calculate time differences for each
            df_before['dt_before'] = (df_before['Start Time'] - df_before['eq_time']).dt.total_seconds().abs()
            df_after['dt_after'] = (df_after['Start Time'] - df_after['eq_time_after']).dt.total_seconds().abs()

            print("Picks with no backward match:", df_before['eq_time'].isna().sum())
            print("Picks with no forward match:", df_after['eq_time_after'].isna().sum())
            print(len(picks))

            # Step 3: Choose the closer one
            nearest_eq = []
            for i in range(len(picks)):
                dt_b = df_before.loc[i, 'dt_before']
                dt_a = df_after.loc[i, 'dt_after']
                
                # Use backward if forward is missing or farther
                if pd.isna(dt_a) or (not pd.isna(dt_b) and dt_b <= dt_a):
                    row = df_before.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time'],
                        'magnitude': row['magnitude'],
                        'latitude': row['latitude'],
                        'longitude': row['longitude'],
                        'depth_km': row['depth_km'],
                        'place': row['place'],
                        'time_diff_seconds': row['dt_before']
                    })
                else:
                    row = df_after.loc[i]
                    nearest_eq.append({
                        'Start Time': row['Start Time'],
                        'abbrv': abbrv,
                        'eq_time': row['eq_time_after'],
                        'magnitude': row['magnitude_after'],
                        'latitude': row['latitude_after'],
                        'longitude': row['longitude_after'],
                        'depth_km': row['depth_km_after'],
                        'place': row['place_after'],
                        'time_diff_seconds': row['dt_after']
                    })

            # Step 4: Convert to DataFrame
            nearest_eq_df = pd.DataFrame(nearest_eq)

            # Add time difference column
            nearest_eq_df['time_since_eq_seconds'] = (nearest_eq_df['Start Time'] - nearest_eq_df['eq_time']).dt.total_seconds()

            # Threshold in seconds (1 day)
            one_day_seconds = 86400
            one_hour_seconds = 3600

            boolarr = abs(nearest_eq_df['time_since_eq_seconds']) < one_hour_seconds
            #boolarr = np.logical_and(nearest_eq_df['time_since_eq_seconds']>-3600,nearest_eq_df['time_since_eq_seconds'] < one_day_seconds)

            eq_trig = np.zeros(len(nearest_eq_df))
            eq_trig[boolarr] = 1

            nearest_eq_df['eq_trig'] = eq_trig
            bootstrapped = []
            for m in tqdm(range(1000)):
                selected = []
                for n in range(len(picks)):
                    rand_index = random.randint(0, len(picks) - 1)
                    selected.append(nearest_eq_df['eq_trig'].iloc[rand_index])
                
                # Count how many are within 1 day
                count_within_hour = np.sum(selected)     
                #print(count_within_hour,len(picks))       

                # Percentage
                percent_within_hour = 100 * (count_within_hour / len(picks))
                #print(percent_within_hour)
                bootstrapped.append(percent_within_hour)


            per_85 = np.percentile(bootstrapped,85)
            per_15 = np.percentile(bootstrapped,15)
            median = np.median(bootstrapped)

            percentage_triggered = pd.DataFrame({'Creepmeter':[abbrv],'per_85':[round(per_85,2)],'per_15':[per_15],'median':[median],'Latitude':lat})


            percentage_triggered_all_eq_boot = pd.concat([percentage_triggered_all_eq_boot,percentage_triggered])
        percentage_triggered_all_eq_boot.sort_values(by='Latitude',inplace=True,ascending=False)
        percentage_triggered_all_eq_boot.reset_index(inplace=True,drop=True)

            
            
            

In [ ]:
percentage_triggered_all_eq_boot

In [ ]:
plt.figure(figsize=(18,6))
ax = plt.subplot(1,1,1)
#plt.plot(percentage_triggered_all['Latitude'],percentage_triggered_all['percentage in day'])
# Add a rectangle: (x, y) = lower left corner; width and height
rect_Hawyard = patches.Rectangle((-0.5, -1), width=4, height=101, linewidth=2, edgecolor=colours[0], 
                                 facecolor=colours[0],alpha=0.2)
rect_Calaveras = patches.Rectangle((3.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[1], 
                                   facecolor=colours[1],alpha=0.2)
rect_Hollister = patches.Rectangle((4.5, -1), width=10, height=101, linewidth=2, edgecolor=colours[2], 
                                   facecolor=colours[2],alpha=0.2)
rect_Parkfield = patches.Rectangle((14.5, -1), width=15, height=101, linewidth=2, edgecolor=colours[4],
                                    facecolor=colours[4],alpha=0.2)
rect_SoCal = patches.Rectangle((29.5, -1), width=7, height=101, linewidth=2, edgecolor=colours[6], 
                               facecolor=colours[6],alpha=0.2)
rect_NS = patches.Rectangle((36.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[7], 
                            facecolor=colours[7],alpha=0.2)
rect_SU = patches.Rectangle((37.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[8], 
                            facecolor=colours[8],alpha=0.2)
rect_IM = patches.Rectangle((38.5, -1), width=1, height=101, linewidth=2, edgecolor=colours[9], 
                            facecolor=colours[9],alpha=0.2)

plt.xticks(np.arange(len(percentage_triggered_all_eq_boot)),percentage_triggered_all_eq_boot['Creepmeter'],rotation=90)
# Add the patch to the Axes
ax.add_patch(rect_Hawyard)
ax.add_patch(rect_Calaveras)
ax.add_patch(rect_Hollister)
ax.add_patch(rect_Parkfield)
ax.add_patch(rect_SoCal)
ax.add_patch(rect_NS)
ax.add_patch(rect_SU)
ax.add_patch(rect_IM)
plt.hlines(0,-0.5,39.5,colors='k')

lower_err = percentage_triggered_all_eq_boot['median'] - percentage_triggered_all_eq_boot['per_15']
upper_err = percentage_triggered_all_eq_boot['per_85'] - percentage_triggered_all_eq_boot['median']

plt.errorbar(np.arange(len(percentage_triggered_all_eq_boot)), percentage_triggered_all_eq_boot['median'], 
             yerr=[lower_err, upper_err], 
             fmt='o', capsize=4,color=colours[3])

plt.plot(np.arange(len(percentage_triggered_all_eq_boot)),percentage_triggered_all_eq_boot['median'],
         color=colours[3],marker='o',label='Earthquake in hour prior')


lower_err = percentage_triggered_all_rainfall_boot['median'] - percentage_triggered_all_rainfall_boot['per_15']
upper_err = percentage_triggered_all_rainfall_boot['per_85'] - percentage_triggered_all_rainfall_boot['median']

plt.errorbar(np.arange(len(percentage_triggered_all_rainfall_boot)), percentage_triggered_all_rainfall_boot['median'], 
             yerr=[lower_err, upper_err], 
             fmt='o', capsize=4,color=colours[5])

plt.plot(np.arange(len(percentage_triggered_all_rainfall_boot)),percentage_triggered_all_rainfall_boot['median'],
         color=colours[5],marker='o',label='Rainfall in hour prior')

#plt.plot(np.arange(len(percentage_triggered_all_rainfall)),percentage_triggered_all_rainfall['percentage in hour'],
#         color=colours[5],marker='^',label='Rainfall in hour prior')
plt.ylabel("% of events")
plt.xlabel('Creepmeter')
plt.legend(loc='upper left', bbox_to_anchor=(0.01, 0.82))
plt.text(1.5,95,'Hayward Fault',verticalalignment='center',horizontalalignment='center')
plt.text(4,95,'Calaveras\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(9,95,'San Andreas Fault: Hollister',verticalalignment='center',horizontalalignment='center')
plt.text(21.5,95,'San Andreas Fault: Parkfield',verticalalignment='center',horizontalalignment='center')
plt.text(33,95,'San Andreas Fault: Salton Sea',verticalalignment='center',horizontalalignment='center')
plt.text(37,95,'North Shoreline\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(38,95,'Superstition Hills\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.text(39,95,'Imperial\nFault', rotation=90,verticalalignment='top',horizontalalignment='center')
plt.xlim([-0.5,39.5])
plt.ylim([-1,100])
plt.tight_layout()
plt.savefig('../../Creep_catalog_stats_figures/Bootstrapped_percentages_events_rain_or_EQ_in_hour_prior.pdf')
plt.show()


In [ ]:
picks['Start Time']+dt.timedelta(days=365)

In [ ]:

from dateutil.relativedelta import relativedelta

def add_one_year(date_value):
  """Adds one year to a date, handling leap years correctly."""
  new_date = date_value + relativedelta(years=1)
  looping = int(str(picks['Start Time'].iloc[-1])[0:4]) - int(str(picks['Start Time'].iloc[0])[0:4])
  if new_date > dt.datetime(2024,1,1):
    new_date = new_date - relativedelta(years=looping+1)
  return new_date

In [ ]:
picks_test_circle = picks.copy(deep=True)

In [ ]:
for i in range(int(str(picks_test_circle['Start Time'].iloc[-1])[0:4]) - int(str(picks_test_circle['Start Time'].iloc[0])[0:4])):
    picks_test_circle['Start Time'] = picks_test_circle['Start Time'].apply(add_one_year)
    print(picks_test_circle)

In [ ]:
float(str(picks['Start Time'].iloc[0])[0:4])